# Capstone Project Part 3

Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#import geocoders
#pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Import neighborhood and borough data from Wikipedia on Toronto, Canada Postal Codes and clean dataframe

In [2]:
# read in table from Wikipedia and change column names
df2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0].rename(index=str, columns={"Postcode":"PostalCode","Neighbourhood":"Neighborhood"})
# remove borough = 'Not assigned'
df2 = df2[df2['Borough'] != 'Not assigned'].reset_index(drop=True) 
# find number of unique postal codes
print('There are {} uniques PostalCodes.'.format(len(df2['PostalCode'].unique())))
# Adjust Not assigned Neighborhood to Borough
df2['Neighborhood'].replace('Not assigned',df2['Borough'], inplace=True)
# Add list of neighborhoods for each PostalCode
df_PostalHood = df2.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', ' .join).reset_index(['PostalCode','Borough'])
df_PostalHood

There are 103 uniques PostalCodes.


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [3]:
df_PostalHood.shape

(103, 3)

Import the Postal Code Coordinates for Toronto then merge with neighborhood data

In [4]:
#Import coordinates to Postal Codes
df_Post_Coords = pd.read_csv('http://cocl.us/Geospatial_data')

In [5]:
# Merge location coordinates to Postal Code file
df_PostalHood_Coords = df_PostalHood.join(df_Post_Coords.set_index('Postal Code'), on='PostalCode', how='inner')

In [6]:
df_PostalHood_Coords

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Change dataframe name to df_Toronto

In [7]:
df_Toronto = df_PostalHood_Coords

Use Foursquare to bring in data on local venues.<br>
First define Foursquare credentials

In [8]:
# Four-Square Credentials
CLIENT_ID = 'JIVUEWG3G04BL5MZ1Y0FCOXQ5LEQDWNWZ0UEIVT5Q1VIHDI0' # your Foursquare ID
CLIENT_SECRET = 'PDOYIMEWSCZSZVZCYM2Y2ID4V4RHQWQ0YCCH1FAHZTSXP344' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Create a function to get nearby venues from location

In [9]:
# Create a function to get nearby venues
LIMIT = 100 # limit number of venues returned by Foursquare API
radius = 800 # define radius to get all venues within one-half mile from zip code location
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Use new function to find venues by each Toronto neighborhood

In [10]:
Toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

Display the first 5 records from the Toronto_venues dataframe.

In [11]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
4,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant


Remove 35 Neighborhods that have fewer than 15 venues

In [12]:
Toronto_venues.groupby('Neighborhood').count().sort_values('Neighborhood Latitude').head(35)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Northwest,1,1,1,1,1,1
Roselawn,3,3,3,3,3,3
Downsview Central,4,4,4,4,4,4
"Downsview, North Park, Upwood Park",4,4,4,4,4,4
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",4,4,4,4,4,4
"Silver Hills, York Mills",4,4,4,4,4,4
"Emery, Humberlea",4,4,4,4,4,4
Willowdale West,5,5,5,5,5,5
"Highland Creek, Rouge Hill, Port Union",5,5,5,5,5,5


In [13]:
Toronto_venues_15 = Toronto_venues.groupby('Neighborhood').filter(lambda x: x['Neighborhood']\
    .count() > 14).reset_index(drop=True)
Toronto_venues_15.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
1,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Starbucks,43.770367,-79.186313,Coffee Shop
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,The Beer Store,43.769793,-79.187505,Beer Store


Create a table with the Neighborhood name and a dummy variable column for each Venue Category.<br>
First find the number of unique Venue Categories

In [14]:
# The Number of Venue Categories
print('There are {} uniques categories.'.format(len(Toronto_venues_15['Venue Category'].unique())))

There are 322 uniques categories.


In [15]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues_15[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues_15['Neighborhood'] 

# move neighborhood column to the first column
#fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
#Toronto_onehot = Toronto_onehot[fixed_columns]

cols = Toronto_onehot.columns.tolist()
column_to_move = "Neighborhood"
new_position = 0

cols.insert(new_position, cols.pop(cols.index(column_to_move)))
Toronto_onehot = Toronto_onehot[cols]

Toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Pl

The number of Venues by Venue Category

In [16]:
Toronto_onehot.shape

(3668, 322)

Calculate the percentage of each Venue Category that occurs in each neighborhood

In [17]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Pl

See the top 5 Venue Categories by neighborhood printed out

In [18]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2                Hotel  0.04
3  American Restaurant  0.04
4           Steakhouse  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.10
1        Skating Rink  0.05
2       Grocery Store  0.05
3  Seafood Restaurant  0.05
4       Shopping Mall  0.05


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.17
1         Pizza Place  0.08
2            Pharmacy  0.08
3           BBQ Joint  0.08
4                Park  0.08


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  0.20
1         Grocery Store  0.20
2  Fast Food Restaurant  0.07
3           Coffee Shop  0.07
4   Fried Chicken Joint  0.07


----Bathurst Manor, Downsview North, Wilson Heights----
            

Create function to create data frame with the top 10 venue categories for each neighborhood

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create table with top 10 venue categories for each neighborhood

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Sushi Restaurant,Theater,Steakhouse,Bar,Gastropub
1,Agincourt,Chinese Restaurant,Grocery Store,Motorcycle Shop,Malay Restaurant,Lounge,Skating Rink,Breakfast Spot,Shopping Mall,Discount Store,Shanghai Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Pizza Place,Pharmacy,Park,BBQ Joint,Fried Chicken Joint,Caribbean Restaurant,Noodle House,Shop & Service,Bubble Tea Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Hardware Store,Fried Chicken Joint,Beer Store,Coffee Shop,Fast Food Restaurant,Liquor Store,Sandwich Place,Video Store
4,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Coffee Shop,Restaurant,Pharmacy,Middle Eastern Restaurant,Shopping Mall,Fast Food Restaurant,Bank,Diner,Bridal Shop


Create clusters based on most common venue types near each neighborhood. <br>
After reviewing different numbers of clusters, I chose 7 as the appropriate number of clusters
<br> <br>
Create clusters and assign to data

In [21]:
# set number of clusters
kclusters = 7

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 5, 3, 4, 3, 5, 1, 2, 6, 6], dtype=int32)

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.dropna(inplace=True)
Toronto_merged.reset_index(inplace=True, drop=True)
Toronto_merged['Cluster Labels']=Toronto_merged['Cluster Labels'].astype(int)
Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4,Pizza Place,Fast Food Restaurant,Coffee Shop,Pharmacy,Supermarket,Filipino Restaurant,Sports Bar,Fried Chicken Joint,Greek Restaurant,Grocery Store
1,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6,Pharmacy,Coffee Shop,Bakery,Indian Restaurant,Yoga Studio,Rental Car Location,Lounge,Burger Joint,Bus Line,Bank
2,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,5,Coffee Shop,Discount Store,Grocery Store,Convenience Store,Chinese Restaurant,Sandwich Place,Department Store,Bus Line,Intersection,Bank
3,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Intersection,Coffee Shop,Diner,Bakery,Metro Station,Park,Bus Station,Convenience Store,Fast Food Restaurant,Soccer Field
4,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,6,Furniture / Home Store,Electronics Store,Indian Restaurant,Spa,American Restaurant,Fast Food Restaurant,Light Rail Station,Latin American Restaurant,Bakery,Bus Station


In [23]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Sushi Restaurant,Theater,Steakhouse,Bar,Gastropub
1,5,Agincourt,Chinese Restaurant,Grocery Store,Motorcycle Shop,Malay Restaurant,Lounge,Skating Rink,Breakfast Spot,Shopping Mall,Discount Store,Shanghai Restaurant
2,3,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Pizza Place,Pharmacy,Park,BBQ Joint,Fried Chicken Joint,Caribbean Restaurant,Noodle House,Shop & Service,Bubble Tea Shop
3,4,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Hardware Store,Fried Chicken Joint,Beer Store,Coffee Shop,Fast Food Restaurant,Liquor Store,Sandwich Place,Video Store
4,3,"Bathurst Manor, Downsview North, Wilson Heights",Pizza Place,Coffee Shop,Restaurant,Pharmacy,Middle Eastern Restaurant,Shopping Mall,Fast Food Restaurant,Bank,Diner,Bridal Shop


Get location coordinates of Toronto for mapping

In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create Map

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Counts of neighborhoods per cluster

In [26]:
neighborhoods_venues_sorted['Cluster Labels'].value_counts()

5    18
2    16
6    10
3     9
1     9
4     4
0     1
Name: Cluster Labels, dtype: int64

Review of clusters

### Cluster 1

In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, \
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0,Intersection,Coffee Shop,Diner,Bakery,Metro Station,Park,Bus Station,Convenience Store,Fast Food Restaurant,Soccer Field


### Cluster 2

In [33]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, \
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Central Toronto,1,Sporting Goods Shop,Coffee Shop,Skating Rink,Italian Restaurant,Café,Restaurant,Diner,Spa,Salon / Barbershop,Rental Car Location
34,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Italian Restaurant,Gastropub,Cosmetics Shop,Breakfast Spot,Seafood Restaurant,Gym
35,Downtown Toronto,1,Café,Hotel,Coffee Shop,Restaurant,Japanese Restaurant,Italian Restaurant,Cocktail Bar,Breakfast Spot,Park,Beer Bar
37,Downtown Toronto,1,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Sushi Restaurant,Theater,Steakhouse,Bar,Gastropub
38,Downtown Toronto,1,Coffee Shop,Hotel,Restaurant,Italian Restaurant,Aquarium,Café,Brewery,Concert Hall,Park,Scenic Lookout
39,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Steakhouse,Deli / Bodega,Italian Restaurant,Concert Hall
40,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Deli / Bodega,American Restaurant,Seafood Restaurant,Gastropub,Steakhouse,Concert Hall
46,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Gastropub,Cocktail Bar,Italian Restaurant,Beer Bar,Japanese Restaurant,Seafood Restaurant
47,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Gastropub,American Restaurant,Deli / Bodega,Steakhouse,Bar,Theater


### Cluster 3

In [34]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, \
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,2,Pub,Bar,Sandwich Place,Breakfast Spot,Coffee Shop,Japanese Restaurant,Bakery,Bank,Ice Cream Shop,Indie Movie Theater
22,East York,2,Café,Coffee Shop,Pizza Place,Gastropub,Greek Restaurant,Fast Food Restaurant,Beer Bar,Breakfast Spot,Asian Restaurant,Beer Store
25,East Toronto,2,Bakery,Café,Bar,Coffee Shop,Sandwich Place,Pizza Place,American Restaurant,Brewery,Sushi Restaurant,Italian Restaurant
26,Central Toronto,2,Pizza Place,Coffee Shop,Italian Restaurant,Café,Park,Food & Drink Shop,Burger Joint,Fast Food Restaurant,Sushi Restaurant,Gym
30,Downtown Toronto,2,Coffee Shop,Restaurant,Café,Pizza Place,Park,Grocery Store,Gastropub,Pub,Pharmacy,Beer Store
32,Downtown Toronto,2,Coffee Shop,Italian Restaurant,Café,Park,Restaurant,Pub,Bakery,Theater,Mexican Restaurant,Performing Arts Venue
42,Central Toronto,2,Coffee Shop,Café,Pizza Place,Italian Restaurant,Sandwich Place,Pub,Vegetarian / Vegan Restaurant,American Restaurant,Tea Room,Restaurant
43,Downtown Toronto,2,Café,Coffee Shop,Restaurant,Bakery,Vegetarian / Vegan Restaurant,Pub,Pizza Place,Bookstore,Bubble Tea Shop,Burrito Place
44,Downtown Toronto,2,Café,Vegetarian / Vegan Restaurant,Bar,Bakery,Vietnamese Restaurant,Mexican Restaurant,Coffee Shop,Dessert Shop,Dumpling Restaurant,Chinese Restaurant
53,West Toronto,2,Coffee Shop,Park,Bakery,Pharmacy,Portuguese Restaurant,Camera Store,Café,Bar,Supermarket,Gym


### Cluster 4

In [35]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, \
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,3,Pizza Place,Coffee Shop,Noodle House,Discount Store,Fast Food Restaurant,Shopping Mall,Seafood Restaurant,Sandwich Place,Bus Stop,Italian Restaurant
8,Scarborough,3,Chinese Restaurant,Pizza Place,Pharmacy,Park,BBQ Joint,Fried Chicken Joint,Caribbean Restaurant,Noodle House,Shop & Service,Bubble Tea Shop
9,Scarborough,3,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Nail Salon,Sandwich Place,Burger Joint,Bakery,Other Great Outdoors,Breakfast Spot,Auto Garage
10,North York,3,Pharmacy,Park,Coffee Shop,Diner,Bakery,Bank,Korean Restaurant,Residential Building (Apartment / Condo),Ice Cream Shop,Housing Development
15,North York,3,Pizza Place,Coffee Shop,Restaurant,Pharmacy,Middle Eastern Restaurant,Shopping Mall,Fast Food Restaurant,Bank,Diner,Bridal Shop
18,East York,3,Pizza Place,Fast Food Restaurant,Brewery,Gastropub,Bank,Bakery,Soccer Stadium,Intersection,Rock Climbing Spot,Athletics & Sports
51,York,3,Pharmacy,Park,Beer Store,Japanese Restaurant,Cosmetics Shop,Café,Discount Store,Mexican Restaurant,Market,Sporting Goods Shop
56,York,3,Brewery,Pizza Place,Coffee Shop,Park,Athletics & Sports,Furniture / Home Store,Beer Store,Bar,Bank,Discount Store
63,Etobicoke,3,Pharmacy,Gym,Convenience Store,Sandwich Place,Bakery,Italian Restaurant,Café,Bus Stop,Dessert Shop,Asian Restaurant


### Cluster 5

In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, \
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Pizza Place,Fast Food Restaurant,Coffee Shop,Pharmacy,Supermarket,Filipino Restaurant,Sports Bar,Fried Chicken Joint,Greek Restaurant,Grocery Store
17,North York,4,Grocery Store,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Hotel,Pharmacy,Sandwich Place,Discount Store,Shopping Mall,Falafel Restaurant
49,North York,4,Grocery Store,Coffee Shop,Fast Food Restaurant,Playground,Pub,Latin American Restaurant,Japanese Restaurant,Shoe Store,Mediterranean Restaurant,Pizza Place
66,Etobicoke,4,Pizza Place,Grocery Store,Hardware Store,Fried Chicken Joint,Beer Store,Coffee Shop,Fast Food Restaurant,Liquor Store,Sandwich Place,Video Store


### Cluster 6

In [37]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, \
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,5,Coffee Shop,Discount Store,Grocery Store,Convenience Store,Chinese Restaurant,Sandwich Place,Department Store,Bus Line,Intersection,Bank
5,Scarborough,5,Middle Eastern Restaurant,Pizza Place,Burger Joint,Pharmacy,Convenience Store,Smoke Shop,Breakfast Spot,Korean Restaurant,Bar,Seafood Restaurant
6,Scarborough,5,Chinese Restaurant,Grocery Store,Motorcycle Shop,Malay Restaurant,Lounge,Skating Rink,Breakfast Spot,Shopping Mall,Discount Store,Shanghai Restaurant
11,North York,5,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Burger Joint,Department Store,Bakery,Restaurant,Bank,Sandwich Place
12,North York,5,Korean Restaurant,Coffee Shop,Park,Bus Line,Café,Sporting Goods Shop,Indian Restaurant,Trail,Japanese Restaurant,Hookah Bar
13,North York,5,Coffee Shop,Ramen Restaurant,Japanese Restaurant,Korean Restaurant,Pizza Place,Restaurant,Sushi Restaurant,Fast Food Restaurant,Sandwich Place,Middle Eastern Restaurant
14,North York,5,Gym,Grocery Store,Beer Store,Japanese Restaurant,Coffee Shop,Asian Restaurant,Office,Bike Shop,Sandwich Place,Café
23,East Toronto,5,Greek Restaurant,Coffee Shop,Pub,Fast Food Restaurant,Spa,Ice Cream Shop,Grocery Store,Café,Pizza Place,Italian Restaurant
28,Central Toronto,5,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Sandwich Place,Fast Food Restaurant,Gym,Restaurant,Indian Restaurant,Pizza Place
29,Central Toronto,5,Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Pizza Place,Spa,Sandwich Place,Pub,Café,Bagel Shop


### Cluster 7

In [38]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, \
                   Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,6,Pharmacy,Coffee Shop,Bakery,Indian Restaurant,Yoga Studio,Rental Car Location,Lounge,Burger Joint,Bus Line,Bank
4,Scarborough,6,Furniture / Home Store,Electronics Store,Indian Restaurant,Spa,American Restaurant,Fast Food Restaurant,Light Rail Station,Latin American Restaurant,Bakery,Bus Station
16,North York,6,Sandwich Place,Furniture / Home Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Sports Bar,Fast Food Restaurant,Bar,Bank,Japanese Restaurant
20,East York,6,Coffee Shop,Electronics Store,Sporting Goods Shop,Furniture / Home Store,Brewery,Sandwich Place,Burger Joint,Restaurant,Sports Bar,Japanese Restaurant
21,East York,6,Indian Restaurant,Afghan Restaurant,Coffee Shop,Yoga Studio,Supermarket,Grocery Store,Fried Chicken Joint,Park,Pharmacy,Pizza Place
24,East Toronto,6,Indian Restaurant,Brewery,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Park,Sandwich Place,Gym,Burger Joint,Café
45,Downtown Toronto,6,Harbor / Marina,Airport Terminal,Boat or Ferry,Airport Service,Sculpture Garden,Airport Lounge,Boutique,Music Venue,Tunnel,Park
48,North York,6,Furniture / Home Store,Clothing Store,Vietnamese Restaurant,Restaurant,Dessert Shop,Coffee Shop,Fried Chicken Joint,Burger Joint,Discount Store,Event Space
61,Mississauga,6,Chinese Restaurant,Coffee Shop,Hotel,Restaurant,Asian Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Fried Chicken Joint,Pizza Place,Sushi Restaurant
62,East Toronto,6,Light Rail Station,Fast Food Restaurant,Bar,Harbor / Marina,Italian Restaurant,Brewery,Grocery Store,Burrito Place,Bakery,Pizza Place
